In [5]:
import sys
import os
# caution: path[0] is reserved for script path (or '' in REPL).
sys.path.insert(1, os.path.abspath('./../src'))

import pandas as pd
import numpy as np
import plot_tools
import data_loader
from energy_channels import EnergyChannel
import datetime

import importlib
importlib.reload(plot_tools)
importlib.reload(data_loader)

<module 'data_loader' from 'c:\\Dev\\Research\\REPT_Enhancements_Tool\\src\\data_loader.py'>

In [9]:
_year = 2012

start = datetime.datetime(year = _year, month = 9, day = 1)
end = datetime.datetime(year = _year+1, month = 1, day = 1)

dataRefsA = data_loader.load_compressed_rept_data(satellite="A", start=start, end=end)
dataRefsB = data_loader.load_compressed_rept_data(satellite="B", start=start, end=end)

FESA_A = dataRefsA["FESA"]
L_A = dataRefsA["L"]
MLT_A = dataRefsA["MLT"]
EPOCH_A = dataRefsA["EPOCH"]
ENERGIES = dataRefsA["ENERGIES"]

FESA_B = dataRefsB["FESA"]
L_B = dataRefsB["L"]
MLT_B = dataRefsB["MLT"]
EPOCH_B = dataRefsB["EPOCH"]
ENERGIES = dataRefsB["ENERGIES"]

FESA_A = FESA_A[:, (ENERGIES < 10)]
FESA_B = FESA_B[:, (ENERGIES < 10)]

Loading REPT data between: 2012-09-01 00:00:00 and 2013-01-01 00:00:00.
Loading : REPT_201209_A.npz
Loading : REPT_201210_A.npz
Loading : REPT_201211_A.npz
Loading : REPT_201212_A.npz
Loading : REPT_201301_A.npz
Loading REPT data between: 2012-09-01 00:00:00 and 2013-01-01 00:00:00.
Loading : REPT_201209_B.npz
Loading : REPT_201210_B.npz
Loading : REPT_201211_B.npz
Loading : REPT_201212_B.npz
Loading : REPT_201301_B.npz


In [10]:
df_to_write = pd.DataFrame()

L_CENTERS = [3.00, 3.25, 3.50, 3.75, 4.00, 4.25, 4.50, 4.75, 5.00, 5.25, 5.50, 5.75, 6.00, 6.25, 6.50, 6.75, 7.00]

for CENTER in L_CENTERS:

    SATIFIES_L_CUT_A = ((CENTER - 0.125) <= L_A) & (L_A < (CENTER + 0.125))
    SATIFIES_L_CUT_B = ((CENTER - 0.125) <= L_B) & (L_B < (CENTER + 0.125))

    FESA_A_DF = pd.DataFrame(FESA_A[SATIFIES_L_CUT_A, :], index=EPOCH_A[SATIFIES_L_CUT_A])
    FESA_B_DF = pd.DataFrame(FESA_B[SATIFIES_L_CUT_B, :], index=EPOCH_B[SATIFIES_L_CUT_B])
    
    
    if np.sum(SATIFIES_L_CUT_A) != 0 or np.sum(SATIFIES_L_CUT_B) != 0:
        
        FESA_AVERAGED = pd.concat([FESA_A_DF, FESA_B_DF], sort=False).sort_index().resample("24h").mean()
        
    elif np.sum(SATIFIES_L_CUT_A) != 0 and np.sum(SATIFIES_L_CUT_B) == 0:
        
        FESA_AVERAGED = FESA_A_DF.sort_index().resample("24h").mean()
        
    elif np.sum(SATIFIES_L_CUT_A) == 0 and np.sum(SATIFIES_L_CUT_B) != 0:
        
        FESA_AVERAGED = FESA_B_DF.sort_index().resample("24h").mean()
        
    else:
        
        FESA_AVERAGED = pd.DataFrame().reindex_like(FESA_A_DF) #Create empty dataframe with same columns
            
    names = {j : f"L={CENTER:2}/E={ENERGIES[j]:2}MeV" for j in range(len(ENERGIES))}
    
    FESA_AVERAGED = FESA_AVERAGED.rename(columns=names)
        
    df_to_write = pd.concat((df_to_write, FESA_AVERAGED), axis=1, join='outer')


In [11]:
savdir = "./../compressed_data/"

df_to_write.to_csv(os.path.join(savdir, f"REPT-DAILY-AVERAGES-{_year}.csv"))

print(df_to_write)

            L=3.0/E=1.8MeV  L=3.0/E=2.1MeV  L=3.0/E=2.6MeV  L=3.0/E=3.4MeV  \
2012-09-01     1155.853883      845.389908      862.780666      915.095607   
2012-09-02      678.079568      459.141329      464.596036      493.643975   
2012-09-03      450.792130      276.510984      280.238402      301.480365   
2012-09-04      669.074044      409.778240      417.505739      467.880080   
2012-09-05      589.980533      347.522356      349.863006      403.448293   
...                    ...             ...             ...             ...   
2012-12-27      598.249659      514.807168      507.784960      735.226547   
2012-12-28      635.984370      553.411489      547.562490      786.742265   
2012-12-29      564.915514      481.380307      472.324813      694.538063   
2012-12-30      618.539293      532.399192      522.270132      766.685764   
2012-12-31      565.573400      483.754404      474.613930      694.357241   

            L=3.0/E=4.2MeV  L=3.0/E=5.2MeV  L=3.0/E=6.3MeV  L=3